In [5]:
import pandas as pd
import os
import numpy as np
import plotly.express as px
import ast
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
from pylab import rcParams

TRAIN = "Train"
EVAL = "Eval"

In [6]:
def embeddings_pca(embeddings_df, n_components=3):
    pca = PCA(n_components=n_components)
    pca.fit(embeddings_df)
    print('Components:', pca.components_)
    print('Explained variance', pca.explained_variance_)
    print('Explained variance ratio', pca.explained_variance_ratio_)
    embeddings = pca.transform(embeddings_df)
    return embeddings

In [22]:
def plot_corr(embeddings, tasks, experiment_name):
    tasks_embeddings = np.concatenate((embeddings, tasks), axis=1)
    columns = ["pca_{}".format(i) for i in range(embeddings.shape[1])]
    columns += ["task_{}".format(i) for i in range(tasks.shape[1])]
    tasks_embeddings_df = pd.DataFrame(data=tasks_embeddings, columns=columns)
    ax = sn.heatmap(tasks_embeddings_df.corr(), annot=True)
    ax.set_ylim(0, 6)
    plt.title(experiment_name)
    plt.savefig('images/{}_corr.png'.format(experiment_name))
    plt.clf()

In [38]:
def plott(embeddings, labels, tasks, title='', task_param='velocity', n_components=3):
    stages = np.array([
        TRAIN if label['stage'] == 'train' else EVAL 
        for label in labels
    ])

    if len(tasks.shape) > 1:
        tasks = np.array([
            float("{0:.2f}".format(
                np.linalg.norm(np.array(task) - np.array([0, 0.85, 0.175]))
            ))
            for task in tasks
        ])

    if n_components == 3:
        fig = px.scatter_3d(embeddings, x=0, y=1, z=2, color=tasks_v, symbol=stages)
    else:
        fig = px.scatter(embeddings, x=0, y=1, color=tasks, symbol=stages)

    fig.update_layout(
        title=title,
        legend=dict(x=-.1, y=-.1)
    )
    fig.show()

In [28]:
def plott2d(embeddings, labels, tasks, title='', task_param='velocity', n_components=3, task_ax=0, pca_ax=0):
    stages = np.array([
        TRAIN if label['stage'] == 'train' else EVAL 
        for label in labels
    ])
    if len(tasks.shape) > 1:
        tasks_v = np.array([
            float("{0:.2f}".format(
                task[task_ax]
            ))
            for task in tasks
        ])

    fig = px.scatter(embeddings, x=pca_ax, y=[0]*len(embeddings), color=tasks_v, symbol=stages)

    fig.update_layout(
        title='{} - Color: Task[{}], X: PCA[{}]'.format(title, task_ax, pca_ax),
        legend=dict(x=-.1, y=-.1)
    )
    #fig.show()
    fig.write_image("images/{}_task{}_pca{}.png".format(title, task_ax, pca_ax))

In [34]:
experiments = []

for dirname, _, _ in os.walk('output'):
    if 'embeddings' in dirname.split('/')[-1]:
        experiments.append(dirname)

experiments.sort()
for i, experiment in enumerate(experiments):
    print(i, experiment)

0 output/cheetah-vel/2020_02_16_00_49_55-intobs/embeddings
1 output/cheetah-vel/2020_02_17_18_09_55-intctx/embeddings
2 output/cheetah-vel/2020_03_03_16_20_26-baseline/embeddings
3 output/cheetah-vel/2020_03_04_17_23_41-baseline/embeddings
4 output/cheetah-vel/2020_03_05_23_41_01-intctx/embeddings
5 output/cheetah-vel/2020_03_06_13_23_33-ctxgrads/embeddings
6 output/cheetah-vel/2020_03_07_23_12_39-intctx/embeddings
7 output/cheetah-vel/2020_03_17_00_33_59-intctx/embeddings
8 output/cheetah-vel/2020_03_26_04_30_37-l2-01/embeddings
9 output/cheetah-vel/2020_03_26_04_34_42-dropout/embeddings
10 output/cheetah-vel/2020_03_27_00_51_58-l2/embeddings
11 output/reach-ml1/2020_04_09_01_49_41-baseline-3adapt/embeddings
12 output/reach-ml1/2020_04_15_22_29_42-baseline/embeddings
13 output/reach-ml1/2020_04_18_23_23_57-baseline-ood/embeddings
14 output/reach-ml1/2020_04_18_23_25_34-rwscale/embeddings
15 output/reach-ml1/2020_04_21_15_12_49-l2-ood/embeddings
16 output/reach-ml1/2020_04_21_15_13_40-

In [42]:
results = [
    #(0, 'Int. Obs 1', 'velocity'),
    ('Int. Ctx', 'output/cheetah-vel/2020_02_17_18_09_55-intctx/embeddings', 'velocity', 2, '00002'),
    ('PEARL', 'output/cheetah-vel/2020_03_03_16_20_26-baseline/embeddings', 'velocity', 2, '00002'),
    #(3, 'PEARL 2', 'velocity'),
    #(4, 'Int. Ctx 2', 'velocity'),
    #(5, 'Ctx Only Grads', 'velocity'),
    #(6, 'Int. Ctx 3', 'velocity'),
    #(7, 'Int. Ctx 4', 'velocity'),
    #(8, 'L2 0.1', 'velocity'),
    #(9, 'Dropout', 'velocity'),    
    ('L2', 'output/cheetah-vel/2020_03_27_00_51_58-l2/embeddings', 'velocity', 2, '00002'),
    #(11, 'L2 0.01 2', 'velocity'),
    #(12, 'PEARL 3 (Small)', 'velocity'),
    #(3, 'Reach Normalized', 'goal')
    #('PEARL', 'output/reach-ml1/2020_04_15_22_29_42-baseline/embeddings', 'goal_pos', '00010'),
    #(13, 'PEARL OOD', 'goal_pos', '00010'),
    #(14, 'Scale', 'goal_pos', '00010'),
    #(15, 'L2 0.01', 'goal_pos', '00010'),
    #(16, 'Int Obs OOD', 'goal_pos', '00010'),
    #(17, 'Int Ctx OOD', 'goal_pos', '00010'), # curiosity (ctx) out of dist on reach
    #('Int Ctx', 'output/reach-ml1/2020_04_23_13_48_31-intctx/embeddings', 'goal_pos', '00010'),
    #('L2 0.01', 'output/reach-ml1/2020_04_25_13_48_34-l2/embeddings', 'goal_pos', '00010'),
    #$(20, 'L2 0.01 OOD', 'goal_pos', '00010'), # L2 reg on encoder out of dist on reach
]

In [43]:
for experiment_name, experiment_path, task_param, n_components, adapt_index in results:
    path = os.path.join(experiment_path, adapt_index)
    embeddings_df = pd.read_csv(os.path.join(path, 'default/tensors.tsv'), sep='\t', header=None)
    labels_df = pd.read_csv(os.path.join(path, 'default/metadata.tsv'), sep='\t', header=None)

    embeddings = embeddings_pca(embeddings_df, n_components=n_components)
    labels = [ast.literal_eval(label.replace("array(", "").replace(", dtype=float32)", "")) for label in labels_df[0].array]
    tasks = np.array([
        [label[task_param]] if type(label[task_param]) is float else label[task_param]
        for label in labels
    ])
    plot_corr(embeddings, tasks, experiment_name)
    plott(embeddings, labels, tasks, title=experiment_name, task_param=task_param, n_components=n_components)
    continue

    for task_ax in range(tasks.shape[1]):
        for pca_ax in range(embeddings.shape[1]):
            plott2d(embeddings, labels, tasks, title=experiment_name, task_param=task_param, task_ax=task_ax, pca_ax=pca_ax)

Components: [[ 0.81558354 -0.26099637  0.16769069  0.13859058 -0.46837684]
 [ 0.0789697  -0.04157958  0.0405794  -0.98825576 -0.11721256]]
Explained variance [3.02519931 1.37581247]
Explained variance ratio [0.64901193 0.29516029]


Components: [[ 0.81317725  0.2027053  -0.0630325   0.37607133 -0.390193  ]
 [-0.58142253  0.30034049 -0.12470709  0.5292475  -0.52544139]]
Explained variance [4.00058857 1.3566778 ]
Explained variance ratio [0.69123217 0.23441034]


Components: [[-0.99879213  0.01218606  0.00951479 -0.04363828 -0.01646049]
 [ 0.01173485 -0.32747976  0.92271523 -0.09025464 -0.18185161]]
Explained variance [1.46406252 0.19296231]
Explained variance ratio [0.69091751 0.09106239]


<Figure size 432x288 with 0 Axes>